In [1]:
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"

In [3]:
def search_song_on_spotify(song_name):
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
    url = f'https://api.spotify.com/v1/search?q={song_name}&type=track'
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        tracks = data['tracks']['items']
        return [(track['id'], track['name'], track['artists'][0]['name']) for track in tracks]
    else:
        print(f"Error fetching data: {response.status_code}")
        return []

In [4]:
def get_audio_features(track_ids):
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
    ids = ','.join(track_ids)
    url = f'https://api.spotify.com/v1/audio-features?ids={ids}'
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data['audio_features']
    else:
        print(f"Error fetching audio features: {response.status_code}")
        return []

In [7]:
def find_most_similar_tracks(track_name):
    # Search for the song on Spotify
    results = search_song_on_spotify(track_name)
    
    if not results:
        print("No tracks found.")
        return
    
    track_ids = [track[0] for track in results]
    
    # Get audio features for the found tracks
    audio_features = get_audio_features(track_ids)
    
    # Filter out None values (in case some features couldn't be retrieved)
    audio_features = [features for features in audio_features if features is not None]
    
    if not audio_features:
        print("No audio features found.")
        return
    
    # Extract some high level features
    feature_vectors = np.array([[features['danceability'], features['energy'], features['valence']] for features in audio_features])
    
    # Calculate similarities with the first track as reference
    similarities = cosine_similarity(feature_vectors)
    
    # Find the indices of the top 10 most similar tracks (excluding itself)
    most_similar_indices = np.argsort(similarities[0][1:])[-10:][::-1] + 1  # +1 because we exclude itself
    
    print(f"Top 10 similar tracks to '{track_name}':")
    
    for index in most_similar_indices:
        similar_track = results[index]
        print(f"- {similar_track[1]} by {similar_track[2]}")

In [9]:
track_name_input = input("Enter a song name to find its most similar tracks on Spotify: ")
find_most_similar_tracks(track_name_input)

Top 10 similar tracks to 'x-tali':
- X-TALI (SLOWED) by MXCCREE
- X-TALI (SUPER SLOWED) by MXCCREE
- X-TALI (SPED UP) by MXCCREE
- X-PRIME by Dj Samir
- X-FAST by ICEDMANE
- EMPIRE by Ogryzek
- X-AURA by Nulteex
- X (Fr Breakfast) by MKTad
- X-talibã by XW1Z
- X-Ecutioner Style (feat. Black Thought) by Linkin Park
